<a href="https://colab.research.google.com/github/Hasan-Mesbaul-Ali-Taher/90DaysOfDevOps/blob/main/lasertagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Note: TPUs for tensorflow 2.0 in colab are still problematic.
%tensorflow_version 1.15.x

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.15.x`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
from google.colab import auth
auth.authenticate_user()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
import datetime
import json
import os
import pprint
import random
import string
import sys
print(tf.__version__)

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)


with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

1.15.0
TPU address is grpc://10.122.251.66:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 54296275588549907),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5753943042210101716),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7365378931066353434),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 15808636614632995787),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 10869153604611798629),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 17736780991318097701),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 597508202454847971),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 7677290435432328383),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 17986874

In [ ]:
!test -d lasertagger || git clone https://github.com/google-research/lasertagger.git
import sys
if not 'lasertagger' in sys.path:
  sys.path += ['lasertagger']

Cloning into 'lasertagger'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 46 (delta 5), reused 41 (delta 3), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [ ]:
!test -d wiki-split || git clone https://github.com/google-research-datasets/wiki-split.git

Cloning into 'wiki-split'...
remote: Enumerating objects: 32, done.
remote: Total 32 (delta 0), reused 0 (delta 0), pack-reused 32
Unpacking objects: 100% (32/32), done.


In [ ]:
!unzip wiki-split/train.tsv.zip
!mv train.tsv ./wiki-split/

Archive:  wiki-split/train.tsv.zip
  inflating: train.tsv               


In [ ]:
#pip install bert, or get it from the repo.
!pip install bert-tensorflow

# import sys

# !test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
# if not 'bert_repo' in sys.path:
#   sys.path += ['bert_repo']

# # import python modules defined by BERT
# import modeling
# import optimization
# import run_classifier
# import run_classifier_with_tfhub
# import tokenization

# # import tfhub 
# import tensorflow_hub as hub

     |████████████████████████████████| 71kB 2.7MB/s 


In [ ]:
TASK = 'wikisplit'

# Bucket to store output data into.
BUCKET = 'contextualexp' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/lasertagger/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

BERT_MODEL = 'cased_L-12_H-768_A-12' #@param {type:"string"}
BERT_DIR = 'gs://cloud-tpu-checkpoints/bert/{}'.format(BERT_MODEL)
#BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

***** Model output directory: gs://contextualexp/lasertagger/wikisplit *****


In [ ]:
LOCAL_OUTPUT_DIR = '/content/output'

%env WIKISPLIT_DIR=/content/wiki-split 
%env LOCAL_OUTPUT_DIR = $LOCAL_OUTPUT_DIR
!mkdir $LOCAL_OUTPUT_DIR
%env OUTPUT_DIR = $OUTPUT_DIR

env: WIKISPLIT_DIR=/content/wiki-split
env: LOCAL_OUTPUT_DIR=/content/output
env: OUTPUT_DIR=gs://contextualexp/lasertagger/wikisplit


In [ ]:
%env BERT_BASE_DIR = /content/$BERT_MODEL
%env BERT_DIR = $BERT_DIR

env: BERT_BASE_DIR=/content/cased_L-12_H-768_A-12
env: BERT_DIR=gs://cloud-tpu-checkpoints/bert/cased_L-12_H-768_A-12


In [ ]:
!python /content/lasertagger/phrase_vocabulary_optimization.py \
  --input_file=/content/wiki-split/train.tsv \
  --input_format=wikisplit \
  --vocabulary_size=500 \
  --max_input_examples=1000000 \
  --output_file={LOCAL_OUTPUT_DIR}/label_map.txt

I1210 23:31:10.132540 140339674630016 phrase_vocabulary_optimization.py:190] 0 examples processed.
I1210 23:31:10.667882 140339674630016 phrase_vocabulary_optimization.py:190] 1000 examples processed.
I1210 23:31:11.168749 140339674630016 phrase_vocabulary_optimization.py:190] 2000 examples processed.
I1210 23:31:11.696878 140339674630016 phrase_vocabulary_optimization.py:190] 3000 examples processed.
I1210 23:31:12.220889 140339674630016 phrase_vocabulary_optimization.py:190] 4000 examples processed.
I1210 23:31:12.734548 140339674630016 phrase_vocabulary_optimization.py:190] 5000 examples processed.
I1210 23:31:13.264718 140339674630016 phrase_vocabulary_optimization.py:190] 6000 examples processed.
I1210 23:31:13.783262 140339674630016 phrase_vocabulary_optimization.py:190] 7000 examples processed.
I1210 23:31:14.299413 140339674630016 phrase_vocabulary_optimization.py:190] 8000 examples processed.
I1210 23:31:14.848136 140339674630016 phrase_vocabulary_optimization.py:190] 9000 exa

In [ ]:
# TODO: Preprocessing this data takes a few minutes -- store / read from cloud bucket.
!python lasertagger/preprocess_main.py \
    --input_file=${WIKISPLIT_DIR}/train.tsv \
    --input_format=wikisplit \
    --output_tfrecord=${LOCAL_OUTPUT_DIR}/train.tf_record \
    --label_map_file=${LOCAL_OUTPUT_DIR}/label_map.txt \
    --vocab_file=${BERT_DIR}/vocab.txt \
    --output_arbitrary_targets_for_infeasible_examples=false


W1210 23:50:31.753316 140163261851520 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

I1210 23:50:33.150728 140163261851520 preprocess_main.py:113] 0 examples processed, 0 converted to tf.Example.
I1210 23:50:36.498775 140163261851520 preprocess_main.py:113] 10000 examples processed, 2644 converted to tf.Example.
I1210 23:50:40.233759 140163261851520 preprocess_main.py:113] 20000 examples processed, 5754 converted to tf.Example.
I1210 23:50:43.923973 140163261851520 preprocess_main.py:113] 30000 examples processed, 8848 converted to tf.Example.
I1210 23:50:47.814198 140163261851520 preprocess_main.py:113] 40000 examples processed, 11893 converted to tf.Example.
I1210 23:50:51.440288 140163261851520 preprocess_main.py:113] 50000 examples processed, 14645 converted to tf.Example.
I1210 23:50:55.167337 140163261851520 preprocess_main.py:113] 60000 examples processed,

In [ ]:
!python lasertagger/preprocess_main.py \
  --input_file=${WIKISPLIT_DIR}/tune.tsv \
  --input_format=wikisplit \
  --output_tfrecord=${LOCAL_OUTPUT_DIR}/tune.tf_record \
  --label_map_file=${LOCAL_OUTPUT_DIR}/label_map.txt \
  --vocab_file=${BERT_DIR}/vocab.txt \
  --output_arbitrary_targets_for_infeasible_examples=true


W1107 11:07:27.758785 139772331399040 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

I1107 11:07:28.929778 139772331399040 preprocess_main.py:113] 0 examples processed, 0 converted to tf.Example.
I1107 11:07:33.731773 139772331399040 preprocess_main.py:121] Done. 5000 examples converted to tf.Example.
I1107 11:07:33.732393 139772331399040 preprocess_main.py:123] Wrote:
/content/output/tune.tf_record
/content/output/tune.tf_record.num_examples.txt


In [ ]:
# Copy local example data to cloud bucket.
filenames = tf.gfile.Glob(LOCAL_OUTPUT_DIR+"/*")
for i, f in enumerate(sorted(filenames)):
  if tf.gfile.Stat(f).is_directory:
    continue
  newpath = os.path.join(OUTPUT_DIR, f.split('/')[-1])
  print("Copying file:" + newpath)
  tf.gfile.Copy(f, newpath)
  #tf.gfile.Copy(f, newpath)

Copying file:gs://contextualexp/lasertagger/wikisplit/label_map.txt


AlreadyExistsError: ignored

In [ ]:
%env NUM_TRAIN_EXAMPLES=310922
%env NUM_EVAL_EXAMPLES=5000
%env CONFIG_FILE=/content/lasertagger/configs/lasertagger_config.json
%env EXPERIMENT=wikisplit_experiment_test
%env TPU_NAME = $TPU_ADDRESS

# TODO: This should write the model checkpoints to TFHub / Google Cloud. 
!python lasertagger/run_lasertagger.py \
  --training_file=${OUTPUT_DIR}/train.tf_record \
  --eval_file=${OUTPUT_DIR}/tune.tf_record \
  --label_map_file=${OUTPUT_DIR}/label_map.txt \
  --model_config_file=${CONFIG_FILE} \
  --output_dir=${OUTPUT_DIR}/models/${EXPERIMENT} \
  --init_checkpoint=${BERT_DIR}/bert_model.ckpt \
  --do_train=true \
  --do_eval=true \
  --train_batch_size=32 \
  --save_checkpoints_steps=2000 \
  --num_train_examples=${NUM_TRAIN_EXAMPLES} \
  --num_eval_examples=${NUM_EVAL_EXAMPLES} \
  --use_tpu=true \
  --tpu_name=${TPU_NAME}
# Note: TPU requires google cloud buckets for the BERT model; load from tensorhub instead.


env: NUM_TRAIN_EXAMPLES=310922
env: NUM_EVAL_EXAMPLES=5000
env: CONFIG_FILE=/content/lasertagger/configs/lasertagger_config.json
env: EXPERIMENT=wikisplit_experiment_test
env: TPU_NAME=grpc://10.122.251.66:8470




W1211 00:06:00.080725 139704924878720 module_wrapper.py:139] From lasertagger/run_lasertagger.py:182: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1211 00:06:00.080967 139704924878720 module_wrapper.py:139] From lasertagger/run_lasertagger.py:182: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1211 00:06:00.081165 139704924878720 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-s

In [ ]:
# TODO: This should export to the task directory in GCS.
# TODO: It seems like the export code should be split from the training code.
!python lasertagger/run_lasertagger.py \
  --label_map_file=${OUTPUT_DIR}/label_map.txt \
  --model_config_file=${CONFIG_FILE} \
  --output_dir=${OUTPUT_DIR}/models/${EXPERIMENT} \
  --do_export=true \
  --export_path=${OUTPUT_DIR}/models/${EXPERIMENT}/export





W1211 00:20:43.414196 140400146028416 module_wrapper.py:139] From lasertagger/run_lasertagger.py:182: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1211 00:20:43.414411 140400146028416 module_wrapper.py:139] From lasertagger/run_lasertagger.py:182: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1211 00:20:43.414611 140400146028416 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1211 00:20:44.618693 14040014

In [ ]:
# Where the model is needs to be standardized - it's currently written to a number in the directory.

ls: cannot access '/content/output/models/wikisplit_experiment_test/export/1573058179': No such file or directory


In [ ]:
%env SAVED_MODEL_DIR=gs://contextualexp/lasertagger/wikisplit/models/wikisplit_experiment_test/export/1573130118
%env PREDICTION_FILE=gs://contextualexp/lasertagger/wikisplit/models/wikisplit_experiment_test/pred.tsv

!export 



env: SAVED_MODEL_DIR=gs://contextualexp/lasertagger/wikisplit/models/wikisplit_experiment_test/export/1573130118
env: PREDICTION_FILE=gs://contextualexp/lasertagger/wikisplit/models/wikisplit_experiment_test/pred.tsv
declare -x BERT_BASE_DIR="/content/cased_L-12_H-768_A-12"
declare -x BERT_DIR="gs://cloud-tpu-checkpoints/bert/cased_L-12_H-768_A-12"
declare -x CLICOLOR="1"
declare -x CLOUDSDK_CONFIG="/content/.config"
declare -x COLAB_GPU="0"
declare -x COLAB_TPU_ADDR="10.122.251.66:8470"
declare -x CONFIG_FILE="/content/lasertagger/configs/lasertagger_config.json"
declare -x CUDA_PKG_VERSION="10-1=10.1.243-1"
declare -x CUDA_VERSION="10.1.243"
declare -x CUDNN_VERSION="7.6.5.32"
declare -x DATALAB_SETTINGS_OVERRIDES="{\"kernelManagerProxyPort\":6000,\"kernelManagerProxyHost\":\"172.28.0.3\",\"jupyterArgs\":[\"--ip=\\\"172.28.0.2\\\"\"]}"
declare -x DEBIAN_FRONTEND="noninteractive"
declare -x ENV="/root/.bashrc"
declare -x EXPERIMENT="wikisplit_experiment_test"
declare -x GCS_READ_CACHE

In [ ]:
!python lasertagger/predict_main.py \
  --input_file=${WIKISPLIT_DIR}/validation.tsv \
  --input_format=wikisplit \
  --output_file=${PREDICTION_FILE} \
  --label_map_file=${OUTPUT_DIR}/label_map.txt \
  --vocab_file=${BERT_DIR}/vocab.txt \
  --saved_model=${SAVED_MODEL_DIR}


W1107 15:30:56.390551 139814862706560 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1107 15:30:56.648868 139814862706560 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

I110

In [ ]:
!python lasertagger/score_main.py --prediction_file=${PREDICTION_FILE}

Traceback (most recent call last):
  File "lasertagger/score_main.py", line 67, in <module>
    app.run(main)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 299, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "lasertagger/score_main.py", line 54, in main
    FLAGS.prediction_file, FLAGS.case_insensitive)
  File "/content/lasertagger/score_lib.py", line 55, in read_data
    for line in f:
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/lib/io/file_io.py", line 220, in __next__
    return self.next()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/lib/io/file_io.py", line 214, in next
    retval = self.readline()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/lib/io/file_io.py", line 178, in readline
    self._preread_check()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/lib/io/f